In [1]:
%load_ext autoreload
%autoreload 2

# ExpectationValue

In [2]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.opflow import PauliSumOp

observable = PauliSumOp.from_list([("XX", 1), ("YY", 2), ("ZZ", 3)])
print("observable\n", observable)

ansatz = RealAmplitudes(num_qubits=2, reps=2)
print("ansatz\n", ansatz)

observable
 1.0 * XX
+ 2.0 * YY
+ 3.0 * ZZ
ansatz
      ┌──────────┐     ┌──────────┐     ┌──────────┐
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├──■──┤ Ry(θ[4]) ├
     ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├
     └──────────┘└───┘└──────────┘└───┘└──────────┘


In [3]:
from qiskit.quantum_info import Statevector

expval = Statevector(ansatz.bind_parameters([0, 1, 1, 2, 3, 5])).expectation_value(
    observable.primitive
)
print(expval.real)

1.8420921273153719


## ExpectationValue class

In [4]:
from qiskit.evaluators import PauliExpectationValue

### AerBackend (QasmSimulator)

In [5]:
from qiskit.providers.aer import AerSimulator

In [6]:
backend = AerSimulator()

expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=array(1.77929688+0.j), variance=array(11.27286911+0.j), confidence_interval=[((1.6185253409360543+0j), (1.9400684090639457+0j))])

In [7]:
# no parameter case

circuit = ansatz.bind_parameters([0, 1, 1, 2, 3, 5])
expval = PauliExpectationValue(circuit, observable, backend=backend)
expval.evaluate()

ExpectationValueResult(value=array(1.78710938+0.j), variance=array(11.09222031+0.j), confidence_interval=[((1.6291003211301207+0j), (1.9451184288698793+0j))])

### Exact simulation by SaveExpectationValueVariance

In [8]:
from qiskit.evaluators import ExactExpectationValue

In [9]:
expval = ExactExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=array(1.84209213), variance=array(6.43276352), confidence_interval=None)

### Mitigator

Readout mitigator classes [#6485](https://github.com/Qiskit/qiskit-terra/pull/6485) is required


In [ ]:
amat = ...

mitigator = CompleteReadoutMitigator(amat)
PauliExpectationValue(ansatz, observable, backend=backend, mitigator=mitigator)
expval.evaluate([0, 1, 1, 2, 3, 5])

### Factory design pattern vs dependency injection